In [1]:
import pandas as pd
import numpy as np
from numpy import asarray

from PIL import Image
from mtcnn.mtcnn import MTCNN

from numpy import load
from numpy import expand_dims
from numpy import savez_compressed
from keras.models import load_model

from tqdm import tqdm
from tqdm import tqdm_notebook

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


## The Main DataFrame:

In [6]:
photo_df_A = pd.read_csv('photo_info_df/photo_dfs_with_images/photo_data_with_image_arrays_A.csv')
photo_df_B = pd.read_csv('photo_info_df/photo_dfs_with_images/photo_data_with_image_arrays_B.csv')
photo_df_C = pd.read_csv('photo_info_df/photo_dfs_with_images/photo_data_with_image_arrays_C.csv')
photo_df_D = pd.read_csv('photo_info_df/photo_dfs_with_images/photo_data_with_image_arrays_D.csv')

In [7]:
photo_df_A.head()

,name,dob,gender,photo_taken,age_when_taken,file_path,face_location,face_score,original_image_array
0,Fred Astaire,1899-05-10,1.0,1968,69,image_data/imdb_data/01/nm0000001_rm124825600_...,[1072.926 161.838 1214.784 303.696],1.459693,[[[ 12 17 13]\n [ 11 16 12]\n [ 13 15 ...
1,Fred Astaire,1899-05-10,1.0,1970,71,image_data/imdb_data/01/nm0000001_rm3343756032...,[477.184 100.352 622.592 245.76 ],2.543198,[[[152 148 103]\n [151 147 102]\n [153 149 1...
2,Fred Astaire,1899-05-10,1.0,1968,69,image_data/imdb_data/01/nm0000001_rm577153792_...,[114.96964309 114.96964309 451.68657236 451.68...,3.455579,[[[ 99 103 114]\n [ 99 103 114]\n [ 99 103 1...
3,Fred Astaire,1899-05-10,1.0,1968,69,image_data/imdb_data/01/nm0000001_rm946909184_...,[622.88550564 424.21750384 844.33900767 645.67...,1.872117,[[[162 179 160]\n [157 172 151]\n [148 162 1...
4,Fred Astaire,1899-05-10,1.0,1968,69,image_data/imdb_data/01/nm0000001_rm980463616_...,[1013.85900236 233.88204221 1201.5861278 42...,1.158766,[[[140 156 81]\n [152 168 97]\n [164 176 1...


In [2]:
#Load the converted metadata files for the images
photo_df = pd.read_csv('photo_info_df/All_photo_data.csv')

In [3]:
photo_df.head()

,name,dob,gender,photo_taken,age_when_taken,file_path,face_location,face_score
0,Fred Astaire,1899-05-10,1.0,1968,69,image_data/imdb_data/01/nm0000001_rm124825600_...,[1072.926 161.838 1214.784 303.696],1.459693
1,Fred Astaire,1899-05-10,1.0,1970,71,image_data/imdb_data/01/nm0000001_rm3343756032...,[477.184 100.352 622.592 245.76 ],2.543198
2,Fred Astaire,1899-05-10,1.0,1968,69,image_data/imdb_data/01/nm0000001_rm577153792_...,[114.96964309 114.96964309 451.68657236 451.68...,3.455579
3,Fred Astaire,1899-05-10,1.0,1968,69,image_data/imdb_data/01/nm0000001_rm946909184_...,[622.88550564 424.21750384 844.33900767 645.67...,1.872117
4,Fred Astaire,1899-05-10,1.0,1968,69,image_data/imdb_data/01/nm0000001_rm980463616_...,[1013.85900236 233.88204221 1201.5861278 42...,1.158766


In [4]:
#There are 372917 total images across 53690 individuals
photo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 372917 entries, 0 to 372916
Data columns (total 8 columns):
name              372917 non-null object
dob               372917 non-null object
gender            372917 non-null float64
photo_taken       372917 non-null int64
age_when_taken    372917 non-null int64
file_path         372917 non-null object
face_location     372917 non-null object
face_score        372917 non-null float64
dtypes: float64(2), int64(2), object(4)
memory usage: 22.8+ MB


In [5]:
photo_df.name.nunique()

53690

In [6]:
photo_df[photo_df['file_path'].str.contains("image_data/imdb_data/", case=False)]

,name,dob,gender,photo_taken,age_when_taken,file_path,face_location,face_score
0,Fred Astaire,1899-05-10,1.0,1968,69,image_data/imdb_data/01/nm0000001_rm124825600_...,[1072.926 161.838 1214.784 303.696],1.459693
1,Fred Astaire,1899-05-10,1.0,1970,71,image_data/imdb_data/01/nm0000001_rm3343756032...,[477.184 100.352 622.592 245.76 ],2.543198
2,Fred Astaire,1899-05-10,1.0,1968,69,image_data/imdb_data/01/nm0000001_rm577153792_...,[114.96964309 114.96964309 451.68657236 451.68...,3.455579
3,Fred Astaire,1899-05-10,1.0,1968,69,image_data/imdb_data/01/nm0000001_rm946909184_...,[622.88550564 424.21750384 844.33900767 645.67...,1.872117
4,Fred Astaire,1899-05-10,1.0,1968,69,image_data/imdb_data/01/nm0000001_rm980463616_...,[1013.85900236 233.88204221 1201.5861278 42...,1.158766
5,Lauren Bacall,1924-09-16,0.0,1991,67,image_data/imdb_data/02/nm0000002_rm1075631616...,[ 686.67682059 458.46721373 1198.10043604 96...,3.490563
6,Lauren Bacall,1924-09-16,0.0,2004,80,image_data/imdb_data/02/nm0000002_rm1346607872...,[ 78.23569407 52.33512938 194.25423516 168.35...,3.479189
7,Lauren Bacall,1924-09-16,0.0,2004,80,image_data/imdb_data/02/nm0000002_rm1363385088...,[ 64.30867104 64.30867104 207.26768087 207.26...,3.870171
8,Lauren Bacall,1924-09-16,0.0,2004,80,image_data/imdb_data/02/nm0000002_rm1447271168...,[242.176 49.664 278.528 86.016],3.472944
9,Lauren Bacall,1924-09-16,0.0,2008,84,image_data/imdb_data/02/nm0000002_rm1624085760...,[108.03894851 28.82624961 158.4253978 79.21...,3.862811


In [7]:
def obtain_image_array(filename):
    
    # load the image
    image = Image.open(filename)
    image = image.convert('RGB')
    image_array = asarray(image)
    
    return image_array

In [8]:
photo_df_A = photo_df[:100000]
photo_df_B = photo_df[100000:200000]
photo_df_C = photo_df[200000:300000]
photo_df_D = photo_df[300000:]

In [9]:
#This block of code obtains the image tensor from the image files (Part A)
tqdm.pandas()
photo_df_A['original_image_array'] = photo_df_A['file_path'].progress_apply(lambda x: obtain_image_array(x))


100%|██████████| 100000/100000 [25:47<00:00, 64.62it/s] 


In [10]:
photo_df_A.to_csv('photo_info_df/photo_data_with_image_arrays_A.csv', index = False)

In [11]:
photo_df_A.head()

,name,dob,gender,photo_taken,age_when_taken,file_path,face_location,face_score,original_image_array
0,Fred Astaire,1899-05-10,1.0,1968,69,image_data/imdb_data/01/nm0000001_rm124825600_...,[1072.926 161.838 1214.784 303.696],1.459693,"[[[12, 17, 13], [11, 16, 12], [13, 15, 12], [1..."
1,Fred Astaire,1899-05-10,1.0,1970,71,image_data/imdb_data/01/nm0000001_rm3343756032...,[477.184 100.352 622.592 245.76 ],2.543198,"[[[152, 148, 103], [151, 147, 102], [153, 149,..."
2,Fred Astaire,1899-05-10,1.0,1968,69,image_data/imdb_data/01/nm0000001_rm577153792_...,[114.96964309 114.96964309 451.68657236 451.68...,3.455579,"[[[99, 103, 114], [99, 103, 114], [99, 103, 11..."
3,Fred Astaire,1899-05-10,1.0,1968,69,image_data/imdb_data/01/nm0000001_rm946909184_...,[622.88550564 424.21750384 844.33900767 645.67...,1.872117,"[[[162, 179, 160], [157, 172, 151], [148, 162,..."
4,Fred Astaire,1899-05-10,1.0,1968,69,image_data/imdb_data/01/nm0000001_rm980463616_...,[1013.85900236 233.88204221 1201.5861278 42...,1.158766,"[[[140, 156, 81], [152, 168, 97], [164, 176, 1..."


In [9]:
#This block of code obtains the image tensor from the image files (Part A)
tqdm.pandas()
photo_df_B['original_image_array'] = photo_df_B['file_path'].progress_apply(lambda x: obtain_image_array(x))


100%|██████████| 100000/100000 [08:44<00:00, 190.77it/s]


In [10]:
photo_df_B.to_csv('photo_info_df/photo_data_with_image_arrays_B.csv', index = False)

In [11]:
photo_df_B.head()

,name,dob,gender,photo_taken,age_when_taken,file_path,face_location,face_score,original_image_array
100000,Rubén Blades,1948-07-16,1.0,1987,39,image_data/imdb_data/52/nm0001952_rm3375795712...,[ 133.12 395.264 1310.72 1572.864],4.095373,"[[[193, 193, 193], [193, 193, 193], [193, 193,..."
100001,Rubén Blades,1948-07-16,1.0,2003,55,image_data/imdb_data/52/nm0001952_rm3472198656...,[300.05000271 151.04900135 745.00500677 596.00...,3.033747,"[[[158, 131, 101], [158, 131, 101], [159, 132,..."
100002,Rubén Blades,1948-07-16,1.0,1987,39,image_data/imdb_data/52/nm0001952_rm4063661568...,[460.8 411.648 606.208 557.056],2.345388,"[[[188, 188, 188], [198, 198, 198], [205, 205,..."
100003,Rubén Blades,1948-07-16,1.0,2003,55,image_data/imdb_data/52/nm0001952_rm4076178432...,[ 679.57785854 453.73457236 1016.29478781 79...,1.900567,"[[[188, 161, 108], [191, 164, 111], [192, 165,..."
100004,Rubén Blades,1948-07-16,1.0,2012,64,image_data/imdb_data/52/nm0001952_rm4138971392...,[458.60634667 247.88710974 772.63720205 561.91...,1.274643,"[[[26, 118, 133], [9, 101, 116], [19, 111, 126..."


In [9]:
#This block of code obtains the image tensor from the image files (Part A)
tqdm.pandas()
photo_df_C['original_image_array'] = photo_df_C['file_path'].progress_apply(lambda x: obtain_image_array(x))


100%|██████████| 100000/100000 [09:54<00:00, 168.17it/s]


In [10]:
photo_df_C.to_csv('photo_info_df/photo_data_with_image_arrays_C.csv', index = False)

In [11]:
photo_df_C.head()

,name,dob,gender,photo_taken,age_when_taken,file_path,face_location,face_score,original_image_array
200000,Autumn Reeser,1980-09-21,0.0,2010,30,image_data/imdb_data/08/nm1032208_rm2165408000...,[401.91418062 88.01908274 479.56195509 165.66...,2.170979,"[[[252, 252, 252], [252, 252, 252], [252, 252,..."
200001,Autumn Reeser,1980-09-21,0.0,2010,30,image_data/imdb_data/08/nm1032208_rm2198962432...,[359.64945851 74.07756296 424.69666517 139.12...,3.233140,"[[[165, 158, 140], [165, 158, 140], [165, 158,..."
200002,Autumn Reeser,1980-09-21,0.0,2010,30,image_data/imdb_data/08/nm1032208_rm2229634304...,[192.15 55.51 314.272 177.632],2.260269,"[[[53, 48, 42], [91, 86, 80], [139, 134, 130],..."
200003,Autumn Reeser,1980-09-21,0.0,2008,28,image_data/imdb_data/08/nm1032208_rm2319495168...,[417.791 321.839 703.648 607.696],3.581139,"[[[45, 40, 8], [47, 43, 8], [49, 45, 8], [51, ..."
200004,Autumn Reeser,1980-09-21,0.0,2005,25,image_data/imdb_data/08/nm1032208_rm2418315008...,[116.204 45.036 155.68 84.512],3.396308,"[[[163, 156, 164], [172, 164, 175], [172, 164,..."


In [9]:
#This block of code obtains the image tensor from the image files (Part A)
tqdm.pandas()
photo_df_D['original_image_array'] = photo_df_D['file_path'].progress_apply(lambda x: obtain_image_array(x))


100%|██████████| 72917/72917 [14:35<00:00, 83.24it/s]  


In [10]:
photo_df_D.to_csv('photo_info_df/photo_data_with_image_arrays_D.csv', index = False)

In [11]:
photo_df_D.head()

,name,dob,gender,photo_taken,age_when_taken,file_path,face_location,face_score,original_image_array
300000,Thomas Craig,1962-12-04,1.0,2008,46,image_data/imdb_data/30/nm0186030_rm3448224000...,[442.28702511 210.58227505 648.77330017 417.06...,2.509885,"[[[57, 52, 46], [56, 51, 45], [54, 49, 43], [5..."
300001,Thomas Craig,1962-12-04,1.0,2008,46,image_data/imdb_data/30/nm0186030_rm3498555648...,[1253.25365032 280.09370007 1459.73992537 48...,2.294894,"[[[19, 16, 1], [18, 15, 0], [18, 15, 0], [17, ..."
300002,Thomas Craig,1962-12-04,1.0,2008,46,image_data/imdb_data/30/nm0186030_rm3532110080...,[510.1953939 303.17238157 677.52985854 470.50...,2.899812,"[[[86, 64, 41], [83, 63, 39], [81, 60, 39], [8..."
300003,Thomas Craig,1962-12-04,1.0,2008,46,image_data/imdb_data/30/nm0186030_rm3699882240...,[608.256 231.424 753.664 376.832],2.532674,"[[[70, 34, 10], [75, 39, 13], [88, 52, 20], [1..."
300004,Thomas Craig,1962-12-04,1.0,2008,46,image_data/imdb_data/30/nm0186030_rm635287808_...,[ 427.90218438 853.75636876 904.94014181 133...,3.279404,"[[[90, 88, 75], [90, 88, 75], [90, 88, 75], [9..."


In [ ]:
#This block combines and saves the dataframes
photo_df = pd.concat([photo_df_A, photo_df_B, photo_df_C, photo_df_D]).reset_index(drop=True)
photo_df.to_csv('photo_info_df/photo_data_with_image_arrays.csv', index = False)

In [ ]:
photo_df.head()

In [ ]:
def resize_picture(filename, dimensions = (160,160), margin = 0):
            
    # load the image
    #image = Image.open(filename)
    #image = image.convert('RGB')
    #image_array = asarray(image)
    
    image_array = filename
    
    #Set a margin boolean and while loop to try margin value
    margin_error = True
    
    while margin_error:
    
        try:
            #Crop the face further with MTCNN
            detector = MTCNN()
    
            #Obtain the first detected face in the cropped face picture
            first_detected_face = detector.detect_faces(image_array)[0]
        
            # get coordinates
            x1, y1, width, height = first_detected_face['box']
            x2, y2 = x1 + width + margin, y1 + height + margin       
            x1 -= margin
            y1 -= margin 
        
            face_array = image_array[y1:y2, x1:x2]
    
        except:
        
            face_array = image_array
        
        try:
            face_array_resized = Image.fromarray(face_array)
            face_array_resized = face_array_resized.resize(dimensions)
            
            margin_error = False
            break
            
        except:
            
            if margin > 0: 
                margin -= 1
            else:
                face_array_resized = Image.fromarray(image_array)
                face_array_resized = face_array_resized.resize(dimensions)
                break
    
    return asarray(face_array_resized)

In [ ]:
picture1 = resize_picture(photo_df.file_path[0], dimensions = (160,160), margin = 10)[1]

In [ ]:
def plot_face(face_data):
    
    # plot face
    plt.axis('off')
    plt.imshow(face_data)

In [ ]:
plot_face(picture1)

In [ ]:
#photo_df[photo_df['file_path'].str.contains("961400_1953-02-05_2011", case=False)]
#photo_df[photo_df['face_score']<=1]
photo_df.file_path[photo_df['face_score'] == photo_df['face_score'].max()]
photo_df.file_path[54153]

In [ ]:
test = photo_df[:100].copy()

In [ ]:
#tqdm.pandas()
#test['original_image_array'] = test['file_path'].progress_apply(lambda x: resize_picture(x, dimensions = (160,160), margin = 8))



In [ ]:
tqdm.pandas()
test['original_image_array'] = test['file_path'].progress_apply(lambda x: obtain_image_array(x))



In [ ]:
tqdm.pandas()
test['fn_resized_array'] = test['original_image_array'].progress_apply(lambda x: resize_picture(x, dimensions = (160,160), margin = 8))



In [ ]:
test.fn_resized_array[3].shape

### Preparation For The Model

In [ ]:
#The following function is from 'Deep Learning for Computer Vision' by Jason Brownlee, Page (508)

# get the face embedding for one face
def get_embedding(model, face_pixels):
    # scale pixel values
    face_pixels = face_pixels.astype('float32')
    # standardize pixel values across channels (global)
    mean, std = face_pixels.mean(), face_pixels.std()
    face_pixels = (face_pixels - mean) / std
    # transform face into one sample
    samples = expand_dims(face_pixels, axis=0)
    # make prediction to get embedding
    yhat = model.predict(samples)
    return yhat[0]

### The Model

In [ ]:
# load the FaceNet model
facenet_model = load_model('FaceNet_Model/facenet_keras.h5')
facenet_model.load_weights('FaceNet_Model/facenet_keras_weights.h5')
print(facenet_model.inputs)
print(facenet_model.outputs)

In [ ]:
get_embedding(facenet_model, picture1)